In [ ]:
import pandas as pd
import re
from rapidfuzz import fuzz

df = pd.read_csv("iplayer_programmes_final.csv")

# Function to clean titles
def clean_title(title):
    title = str(title)
    # Remove episodes with Signed, trail removed, or short in the title
    if 'SIGNED' in title.upper() or 'TRAIL_REMOVED' in title.upper() or 'SHORT' in title.upper():
        return None
    
    # Clean up the title 
    title = re.sub(r'\b\d{2}/\d{2}\b', '', title)
    title = re.sub(r"(S\d+|Season \d+|Christmas special.*|Live.*|Reunion.*|Special.*|\[.*?\])", "", title, flags=re.IGNORECASE)
    title = re.sub(r"[^\w\s']", "", title)
    title = re.sub(r"\d+", "", title)
    return title.strip()

df['Cleaned Title'] = df['Programme / Series Title'].apply(clean_title)
df = df.dropna(subset=['Cleaned Title'])

# Function to create a "clean title" based on the first 4 words
def base_title(title, num_words=4):
    return ' '.join(title.split()[:num_words])

df['Base Title'] = df['Cleaned Title'].apply(lambda x: base_title(x))

# Function to group titles based on fuzzy matching
def group_titles(titles, threshold=85):
    groups = {}
    for title in titles:
        matched = False
        for key in groups:
            score = fuzz.ratio(title.lower(), key.lower())
            if score >= threshold:
                groups[key].append(title)
                matched = True
                break
        if not matched:
            groups[title] = [title]
    return groups

unique_base_titles = df['Base Title'].dropna().unique()
grouped = group_titles(unique_base_titles)

# Create a mapping of parent programmes to their variants
parent_map = {}
for parent, variants in grouped.items():
    for variant in variants:
        parent_map[variant] = parent

# Map the parent programme to each variant in the DataFrame
df['Parent Programme'] = df['Base Title'].map(parent_map)


df.to_csv("iplayer_programmes_with_parent.csv", index=False)


In [13]:

df.head()

,Program ID,Programme / Series Title,TX Type,EdCat 1,EdCat 2,EdCat 3,Min of TX Date,Max of Publ End Date,Cleaned Title,Base Title,Parent Programme
6,181447,THE SARAH JANE ADVENTURES '1' [CHILDRENS],Catalogue Transmission,Childrens,Drama,Various / Other,28/11/2022,28/11/2025,THE SARAH JANE ADVENTURES '',THE SARAH JANE ADVENTURES,THE SARAH JANE ADVENTURES
19,181938,VOYAGES OF DISCOVERY (CC) [PRE],Catch-Up Transmission,Factual,History,World,17/08/2024,14/10/2024,VOYAGES OF DISCOVERY CC,VOYAGES OF DISCOVERY CC,VOYAGES OF DISCOVERY CC
23,182038,AMAZON BRUCE PARRY LANDMARK [PRE],Catalogue Transmission,Factual,Documentary - World,Adventure/Survival,23/03/2024,11/05/2025,AMAZON BRUCE PARRY LANDMARK,AMAZON BRUCE PARRY LANDMARK,AMAZON BRUCE PARRY LANDMARK
27,182142,Top Gear Series 12 [PRE],Catalogue Transmission,Factual Entertainment,Transport/Motoring,NaN,05/05/2021,05/05/2027,Top Gear Series,Top Gear Series,Top Gear Series
29,182197,Horrible Histories 1,Catalogue Transmission,Childrens,Factual,Various / Other,17/05/2024,20/07/2026,Horrible Histories,Horrible Histories,Horrible Histories
